# Basic Example: Prompt+Model+OutputParser

- Author: [ChangJun Lee](https://www.linkedin.com/in/cjleeno1/)
- Peer Review: [Erika Park](https://www.linkedin.com/in/yeonseo-park-094193198/), [Wooseok Jeong](https://github.com/jeong-wooseok)
- Proofread : [Q0211](https://github.com/Q0211)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/01-Basic/06-LangChain-Expression-Language(LCEL).ipynb)
[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/01-Basic/06-LangChain-Expression-Language(LCEL).ipynb)


## Overview

The most fundamental and commonly used case involves linking a prompt template with a model. To illustrate how this works, let us create a chain that asks for the capital cities of various countries.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Utilizing Prompt Templates](#utilizing-prompt-templates)
- [Chain Creation](#chain-creation)

### References

- [LangChain ChatOpenAI API reference](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)
- [LangChain Core Output Parsers](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html#)
- [Python List Tutorial](https://docs.python.org/3.13/tutorial/datastructures.html)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "",
    }
)

Environment variables have been set successfully.


---
You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it. 

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [1]:
# Configuration File for Managing API Key as an Environment Variable
from dotenv import load_dotenv

# Load API KEY Information
load_dotenv(override=True)

True

In [2]:
# Set up LangSmith tracking: https://smith.langchain.com
from langsmith import utils

utils.tracing_is_enabled()

True

## Utilizing Prompt Templates

```PromptTemplate```

- A prompt template is used to create a complete prompt string by incorporating the user's input variables.
- Usage
  - ```template```: A template string is a predefined format where curly braces '{}' are used to represent variables.

  - ```input_variables```: The names of the variables to be inserted within the curly braces are defined as a list.

```input_variables```

- ```input_variables``` is a list that defines the names of the variables used in the ```PromptTemplate```.

In [3]:
from langchain_core.prompts import PromptTemplate
import os
import httpx

AZURE_OPENAI_ENDPOINT = os.environ.get('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.environ.get('AZURE_OPENAI_API_VERSION')
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get('AZURE_OPENAI_DEPLOYMENT_NAME')
AZURE_OPENAI_API_KEY = os.environ.get('AZURE_OPENAI_API_KEY')

httpx_client = httpx.Client(http2=True, verify=False)

The ```from_template()``` method is used to create a ```PromptTemplate``` object.

In [4]:
# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method.
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}?')

In [5]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

'What is the capital of Korea?'

In [6]:
# Generate the prompt.
prompt = prompt_template.format(country="USA")
prompt

'What is the capital of USA?'

In [7]:
from langchain_openai.chat_models import AzureChatOpenAI

model = AzureChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.1,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    openai_api_version = AZURE_OPENAI_API_VERSION,
    deployment_name = AZURE_OPENAI_DEPLOYMENT_NAME,
    openai_api_key = AZURE_OPENAI_API_KEY,
    http_client = httpx_client,
)

## Chain Creation

### LCEL (LangChain Expression Language)

Here, we use LCEL to combine various components into a single chain.

![lcel.png](./assets/02-langchain-expression-language.png)

```
chain = prompt | model | output_parser
```

The ```|``` symbol works similarly to the [Unix pipe operator](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>), linking different components and passing the output of one component as the input to the next.

In this chain, user input is passed to the prompt template, and the output from the prompt template is then forwarded to the model. By examining each component individually, you can understand what happens at each step.

In [9]:
# Create the prompt as a `PromptTemplate` object.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")


# Combine the prompt and model into a chain
chain = prompt | model
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Please explain {topic} in simple terms.')
| AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001393C0AD450>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001393C4F81D0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x000001393A830A10>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x000001393C0AE410>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), http_client=<httpx.Client object at 0x000001393A58B550>, disabled_params={'parallel_tool_calls': None}, azure_endpoint='https://pstestopenaidply-3icksnah2q6ks.openai.azure.com/', deployment_name='pstestopenaidply-3icksnah2q6ks', openai_api_version='2024-12-01-preview', openai_api_type='azure')

### Calling ```invoke()```

- Input values are provided in the form of a Python dictionary (key-value pairs).  
- When calling the ```invoke()``` function, these input values are passed as arguments.

In [11]:
# Set the topic in the `input` dictionary to 'The Principles of Learning in Artificial Intelligence Models'.
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [14]:
# Connect the `prompt` object and the `model` object using the pipe (`|`) operator.
# Use the `invoke` method to pass the `input`.
# This will return the message generated by the AI model.
result = chain.invoke(input)
print(result.content, end="", flush=True)

Sure! The principles of learning in artificial intelligence (AI) models can be understood as the basic ideas that guide how these models learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data**: AI models learn from data. The more relevant and high-quality data they have, the better they can learn. Think of data as the "food" that helps the model grow and improve.

2. **Patterns**: AI models look for patterns in the data. Just like humans recognize trends or similarities in experiences, AI identifies relationships and structures in the data to make predictions or decisions.

3. **Feedback**: Learning often involves feedback. When an AI model makes a prediction, it can receive feedback on whether it was correct or not. This feedback helps the model adjust and improve its future predictions.

4. **Generalization**: A good AI model should not just memorize the data it was trained on but should also be able to apply what 

Below is an example of outputting a streaming response:

In [14]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token.content, end="", flush=True)

Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:

1. **Data is Key**: AI models learn from data. The more quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can do.

2. **Learning from Examples**: AI models learn by looking at examples. For instance, if you show an AI many pictures of cats and dogs, it can learn to tell the difference between them. This is similar to how humans learn by observing and practicing.

3. **Feedback Loop**: AI models improve through feedback. When they make mistakes, they can adjust their understanding based on the corrections. This is like a teacher giving feedback to a student to help them learn from their errors.

4. **Generalization**: AI models aim to generalize f

### Output Parser

An **Output Parser** is a tool designed to transform or process the responses from an AI model into a specific format. Since the model's output is typically provided as free-form text, an **Output Parser** is essential to convert it into a structured format or extract the required data.


In [16]:
from langchain_core.output_parsers import StrOutputParser

output_parser = (
    StrOutputParser()
)  # Directly returns the model's response as a string without modification.

An output parser is added to the chain.

In [17]:
# A processing chain is constructed by connecting the prompt, model, and output parser.
chain = prompt | model | output_parser

In [18]:
# Use the invoke method of the chain object to pass the input
chain.invoke(input)

'Sure! The Principles of Learning in Artificial Intelligence (AI) Models can be understood as the basic ideas that guide how AI systems learn from data and improve their performance over time. Here are some key principles explained in simple terms:\n\n1. **Data is Key**: AI models learn from data. The more high-quality data they have, the better they can learn. Think of it like a student studying for a test; the more information they have, the better they can prepare.\n\n2. **Learning from Examples**: AI models often learn by looking at examples. For instance, if you want an AI to recognize cats in pictures, you show it many pictures of cats and non-cats. Over time, it learns to identify the features that make a cat a cat.\n\n3. **Feedback Loop**: AI models improve through feedback. When they make a mistake, they receive feedback (like a teacher correcting a student), which helps them adjust and do better next time. This is often done through a process called "training."\n\n4. **Genera

In [19]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

Sure! The principles of learning in artificial intelligence (AI) models can be understood through a few key concepts. Here’s a simple breakdown:

1. **Data**: AI models learn from data. Just like humans learn from experiences, AI systems learn from examples. The more relevant data they have, the better they can learn.

2. **Training**: This is the process where the AI model is exposed to data. During training, the model tries to understand patterns and relationships in the data. It adjusts its internal settings (called parameters) to improve its predictions or decisions.

3. **Feedback**: After making predictions, the model receives feedback on how well it did. This feedback helps the model learn from its mistakes. For example, if it predicts something incorrectly, it adjusts its parameters to avoid making the same mistake in the future.

4. **Generalization**: The goal of an AI model is not just to memorize the training data but to generalize. This means it should be able to make accu

### Applying and Modifying Templates

- The prompt content below can be **modified** as needed for testing purposes.  
- The ```model_name``` can also be adjusted for testing.

In [20]:
template = """
You are a seasoned English teacher with 10 years of experience. Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#SITUATION:
{question}

#FORMAT:
- Dialogue in English:
- Explanation of the Dialogue: 
"""

# Generate the prompt using the PromptTemplate
prompt = PromptTemplate.from_template(template)

model = AzureChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.1,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    openai_api_version = AZURE_OPENAI_API_VERSION,
    deployment_name = AZURE_OPENAI_DEPLOYMENT_NAME,
    openai_api_key = AZURE_OPENAI_API_KEY,
    http_client = httpx_client,
)

# Initialize the ChatOpenAI model.
# model = ChatOpenAI(model_name="gpt-4o-mini")

# Initialize the string output parser.
output_parser = StrOutputParser()

In [21]:
# Construct the chain.
chain = prompt | model | output_parser

In [22]:
# Execute the completed Chain to obtain a response.
print(chain.invoke({"question": "I want to go to a restaurant and order food."}))

- Dialogue in English:
**Customer:** Hi there! Could I see the menu, please?  
**Waiter:** Of course! Here you go. Do you have any questions about the menu?  
**Customer:** Yes, I’m curious about the pasta dishes. Which one do you recommend?  
**Waiter:** Our spaghetti carbonara is very popular. It’s made with fresh ingredients and has a creamy sauce.  
**Customer:** That sounds delicious! I’ll have the spaghetti carbonara, please. And can I get a side salad with that?  
**Waiter:** Absolutely! Would you like any dressing for your salad?  
**Customer:** Yes, I’d like balsamic vinaigrette, please.  
**Waiter:** Great choice! Would you like anything to drink?  
**Customer:** Just water, please.  
**Waiter:** Perfect! I’ll get that order in for you right away.  
**Customer:** Thank you!

- Explanation of the Dialogue: 
In this conversation, the customer initiates the interaction by asking for the menu, which is a common first step when dining out. The waiter responds politely and offers a

In [23]:
# Execute the completed Chain to obtain a response
# Request for Streaming Output
answer = chain.stream({"question": "I want to go to a restaurant and order food."})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:  
**Customer:** Hi there! Could I see the menu, please?  
**Waiter:** Of course! Here you go. Do you have any questions about the menu?  
**Customer:** Yes, I’m curious about the pasta dishes. Which one do you recommend?  
**Waiter:** Our spaghetti carbonara is very popular. It’s made with fresh ingredients and has a creamy sauce.  
**Customer:** That sounds delicious! I’ll have the spaghetti carbonara, please. Can I also get a side salad?  
**Waiter:** Absolutely! Would you like any dressing with your salad?  
**Customer:** Yes, please. I’d like balsamic vinaigrette.  
**Waiter:** Great choice! Would you like anything to drink?  
**Customer:** Just water, please.  
**Waiter:** Perfect! I’ll get that order in for you right away.  
**Customer:** Thank you!  

- Explanation of the Dialogue:  
In this conversation, the customer initiates the interaction by asking for the menu, indicating their desire to order food. The waiter responds politely and offers assistance,

In [23]:
# This time, set the question to 'Ordering Pizza in the US' and execute it.
# Request for Streaming Output
answer = chain.stream({"question": "Ordering Pizza in the US"})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- Dialogue in English:

**Customer:** Hi there! I’d like to order a pizza, please.

**Pizza Server:** Of course! What size would you like? We have small, medium, large, and extra-large.

**Customer:** I’ll take a large, please. 

**Pizza Server:** Great choice! What type of pizza do you want? We have pepperoni, cheese, veggie, and a few specialty pizzas.

**Customer:** I’ll go with pepperoni. Can I add extra cheese?

**Pizza Server:** Absolutely! Extra cheese on a large pepperoni pizza. Would you like anything else? 

**Customer:** Yes, can I also get a side of garlic bread and a two-liter soda?

**Pizza Server:** Sure! We have a few soda options. We have cola, diet cola, lemon-lime, and root beer. Which one would you like?

**Customer:** I’ll take a cola, please.

**Pizza Server:** Great! So that’s one large pepperoni pizza with extra cheese, a side of garlic bread, and a cola. Would you like to add any dipping sauces?

**Customer:** Yes, please! Can I get a marinara sauce and a ranch